# 02.A: Working with Datasets

It's important to have a clear and sensible way of representing the datasets that learning algorithms train on. A dataset consists of $n$ examples. Each example consists of $m$ features. This makes $m$ the number dimensions the dataset has. In supervised learning, the dataset is a matrix like this:

$\boldsymbol{D} =\left[\begin{array}{cccccc} 
  x_1^{(1)} & x_2^{(1)} & x_3^{(1)} & \cdots & x_m^{(1)} & y^{(1)}\\ 
  x_1^{(2)} & x_2^{(2)} & x_3^{(2)} & \cdots & x_m^{(2)} & y^{(2)}\\
  x_1^{(3)} & x_2^{(3)} & x_3^{(3)} & \cdots & x_m^{(3)} & y^{(3)}\\
  \vdots    & \vdots    & \vdots    & \cdots & \vdots & \vdots \\
  x_1^{(n)} & x_2^{(n)} & x_3^{(n)} & \cdots & x_m^{(n)} & y^{(n)}
\end{array}\right]$

Each row of this matrix is an example consisting of the $m$ features plus the target label as the last element in the row. In other words, $\boldsymbol{D}$ consists of both the input matrix $\boldsymbol{X}$ and target vector $y$, where: 

$\boldsymbol{X} =\left[\begin{array}{ccccc} 
  x_1^{(1)} & x_2^{(1)} & x_3^{(1)} & \cdots & x_m^{(1)}\\ 
  x_1^{(2)} & x_2^{(2)} & x_3^{(2)} & \cdots & x_m^{(2)}\\
  x_1^{(3)} & x_2^{(3)} & x_3^{(3)} & \cdots & x_m^{(3)}\\
  \vdots    & \vdots    & \vdots    & \cdots & \vdots \\
  x_1^{(n)} & x_2^{(n)} & x_3^{(n)} & \cdots & x_m^{(n)}
\end{array}\right]$

and

$\boldsymbol{y} =\left[\begin{array}{c} 
  y^{(1)}\\ 
  y^{(2)}\\
  y^{(3)}\\
  \vdots \\
  y^{(n)}
\end{array}\right]$

For unsupervised learning, $\boldsymbol{D}$ is the same as $\boldsymbol{X}$. Here is a class named `DataSet` to represent datasets. It uses pandas' DataFrame.

Another name for $X$ is `inputs`, and another name for $y$ is `target`. In addition, features have names. Let's put all of this together in a class that we will be using in subsequent weeks.

In [211]:
import numpy as np
import pandas as pd

#testing this repository

class DataSet:
    """
    A dataset for a machine learning problem. A dataset d has the following properties:
    d.examples   A list of examples. Each one contains both the features and the target.
    d.features   An array of the of feature names.
    d.target     An m by 1 array containing the values of y
    d.y          Same as d.target
    d.inputs     An n by m array containing the values of X
    d.X          Same as d.inputs
    d.N          Number of examples
    d.M          Number of dimensions
    d.name       The name of the data set (for output display only)
    
    """
    def __init__(self, data, features=None, y=None, name=None):
        """
        If y is True, the data contains the target as the last column
        If y is None or False, No target is available
        Else y is an array to be added as the last column of the examples  dataframe
        """
        self.__name = name
        if isinstance(data, pd.DataFrame):
            self.__examples = data
        else:
            self.__examples = pd.DataFrame(data, columns=features)
            
        if y is True:
            self.__examples.columns = [*self.__examples.columns[:-1], 'y']
        elif y is not False and y is not None:
            self.__examples['y'] = y
    
    @property
    def examples(self):
        return self.__examples
    
    @property
    def features(self):
        return self.__examples.columns[:-1].values
    
    @property
    def target(self):
        if 'y' in self.__examples.columns:
            return self.__examples['y'].values.reshape(self.N, 1)
        return None
    
    @property
    def y(self):
        return self.target
    
    @property
    def inputs(self):
        return self.__examples.iloc[:, :-1].values
    
    @property
    def X(self):
        return self.inputs
    
    @property
    def name(self):
        return self.__name
    
    @property
    def N(self):
        return self.__examples.shape[0]
    
    @property
    def M(self):
        return self.inputs.shape[1]
    
    def shuffled(self, random_state=None):
        rgen = np.random.RandomState(random_state)
        indexes = np.arange(self.N)
        rgen.shuffle(indexes)
        return DataSet(self.__examples.iloc[indexes])
    
    def train_test_split(self, start=0, end=None, test_portion=None, shuffle=False, random_state=None):
        """
        Splits the dataset into a training set and atest set. 
        If test_portion is specified, return that portion of the dataset as test 
        and the rest as training. 
        Otherwise, return the examples between start and end as test and the 
        rest as training.
        """
        indexes = np.arange(self.N)
        if shuffle is True:
            rgen = np.random.RandomState(random_state)
            rgen.shuffle(indexes)

        if test_portion is None:
            end = end or self.N
        else:
            if not isinstance(test_portion, float) or test_portion < 0 or test_portion > 1:
                raise TypeError("Only fractions between ]0,1[ are allowed")

            start = self.N - int(self.N * test_portion)
            end = self.N

        test = DataSet(self.examples.iloc[indexes[range(start, end)]])
        train = DataSet(pd.concat([self.examples.iloc[indexes[range(start)]], 
                                      self.examples.iloc[indexes[range(end, self.N)]]], axis=0))    
        return train, test
    def train_validation_test_split(self, training=.5, validation=.25, test = .25, shuffle = False):
        
        deltaOne=round(self.N*training) #get numerical amount of training data
        deltaTwo=round(self.N*validation) #get numerical amount of validation group
                                         #the remaining amount will make up the test data group; 
                                         #not needed but can be insinuated; = self.N-round(self.N*training)-round(self.N*validation)
        if (deltaOne+deltaTwo>self.N): #catch extremely rare but possible rounding error.
            raise TypeError("rounding has caused critical error, should only happen when selecting miniscul sections.  Raise the decimal values and try again")

        Interim, Td = self.train_test_split(start=0, end = deltaOne, shuffle=shuffle) #grab the training data Td
        Te, Vd = Interim.train_test_split(end= deltaTwo, shuffle=shuffle) #split the remaining into Validation data Vd, and test data Te

        return Td, Vd, Te



    def __repr__(self):
        return repr(self.examples)

This class has a couple of properties including `name` (informational), `features` (the names of the features), `inputs`, `target`, `X`, `y`, `N` (number of examples), `M` (number of dimensions).

A DataSet object is created using a NumPy array or a Pandas dataframe. If it is a NumPy array, the class uses it to create a Pandas dataframe. The dataframe storing the data can be retrieved back using the `examples` property.


Let's test this class by creating a $27 \times 3$ input data and a separate $y$ column.

In [212]:
ds = DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")

ds

     x1   x2         x1  y
0   3.0  1.0   7.715742  1
1   5.0  3.0   7.561542  0
2   2.0  8.0  12.632742  0
3   7.0  8.0  10.474343  0
4   4.0  1.0   7.338594  0
5   3.0  7.0  11.478981  0
6   5.0  4.0  11.014609  0
7   2.0  1.0   8.173837  0
8   3.0  6.0  10.602692  0
9   5.0  4.0   7.260641  1
10  2.0  3.0  13.751146  1
11  5.0  3.0  12.741618  0
12  6.0  6.0   7.716400  1
13  6.0  5.0   9.361649  1
14  8.0  5.0  11.804509  0
15  8.0  6.0   8.805556  0
16  6.0  7.0   7.265146  0
17  3.0  1.0   7.183198  0
18  8.0  2.0  10.699841  0
19  5.0  4.0  11.029659  0
20  7.0  7.0  12.894391  1
21  3.0  5.0  11.950252  0
22  7.0  7.0  14.118866  1
23  7.0  1.0   7.594645  1
24  4.0  2.0   9.619569  1
25  4.0  8.0  11.004657  1
26  2.0  8.0   4.700345  1

In [213]:
ds.examples

,x1,x2,x1,y
0,3.0,1.0,7.715742,1
1,5.0,3.0,7.561542,0
2,2.0,8.0,12.632742,0
3,7.0,8.0,10.474343,0
4,4.0,1.0,7.338594,0
5,3.0,7.0,11.478981,0
6,5.0,4.0,11.014609,0
7,2.0,1.0,8.173837,0
8,3.0,6.0,10.602692,0
9,5.0,4.0,7.260641,1


In [214]:
ds.features

array(['x1', 'x2', 'x1'], dtype=object)

In [215]:
ds.target 

array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [216]:
ds.y 

array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [217]:
ds.inputs

array([[ 3.        ,  1.        ,  7.71574184],
       [ 5.        ,  3.        ,  7.5615419 ],
       [ 2.        ,  8.        , 12.6327416 ],
       [ 7.        ,  8.        , 10.47434254],
       [ 4.        ,  1.        ,  7.33859427],
       [ 3.        ,  7.        , 11.47898078],
       [ 5.        ,  4.        , 11.01460914],
       [ 2.        ,  1.        ,  8.17383663],
       [ 3.        ,  6.        , 10.60269195],
       [ 5.        ,  4.        ,  7.2606414 ],
       [ 2.        ,  3.        , 13.75114607],
       [ 5.        ,  3.        , 12.74161765],
       [ 6.        ,  6.        ,  7.71639971],
       [ 6.        ,  5.        ,  9.3616491 ],
       [ 8.        ,  5.        , 11.80450933],
       [ 8.        ,  6.        ,  8.80555593],
       [ 6.        ,  7.        ,  7.26514567],
       [ 3.        ,  1.        ,  7.18319788],
       [ 8.        ,  2.        , 10.69984103],
       [ 5.        ,  4.        , 11.02965935],
       [ 7.        ,  7.        , 12.894

In [218]:
ds.X

array([[ 3.        ,  1.        ,  7.71574184],
       [ 5.        ,  3.        ,  7.5615419 ],
       [ 2.        ,  8.        , 12.6327416 ],
       [ 7.        ,  8.        , 10.47434254],
       [ 4.        ,  1.        ,  7.33859427],
       [ 3.        ,  7.        , 11.47898078],
       [ 5.        ,  4.        , 11.01460914],
       [ 2.        ,  1.        ,  8.17383663],
       [ 3.        ,  6.        , 10.60269195],
       [ 5.        ,  4.        ,  7.2606414 ],
       [ 2.        ,  3.        , 13.75114607],
       [ 5.        ,  3.        , 12.74161765],
       [ 6.        ,  6.        ,  7.71639971],
       [ 6.        ,  5.        ,  9.3616491 ],
       [ 8.        ,  5.        , 11.80450933],
       [ 8.        ,  6.        ,  8.80555593],
       [ 6.        ,  7.        ,  7.26514567],
       [ 3.        ,  1.        ,  7.18319788],
       [ 8.        ,  2.        , 10.69984103],
       [ 5.        ,  4.        , 11.02965935],
       [ 7.        ,  7.        , 12.894

In [219]:
ds.name

'Sample Data'

In [220]:
ds.N

27

In [221]:
ds.M

3

## Shuffling

The above class also supports a few useful methods. One such method is for shuffling the data, which we do often before training. This method returns a new DataSet instance with the shuffled data. Here is how this method is implemented:

```python
    ...
    def shuffled(self, random_state=None):
        rgen = np.random.RandomState(random_state)
        indexes = np.arange(self.N)
        rgen.shuffle(indexes)
        return DataSet(self.__examples.iloc[indexes])
   ...
```

Here is an example using this function.

In [222]:
ds = DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")


ds.shuffled()

     x1   x2         x1  y
23  3.0  5.0   8.438144  1
22  8.0  1.0  10.613207  1
4   3.0  2.0   5.785744  0
12  3.0  3.0   9.519482  0
18  6.0  5.0  10.357077  0
24  6.0  4.0   7.041312  1
17  5.0  4.0  10.606033  0
13  5.0  8.0   8.673531  0
9   6.0  6.0   8.758025  1
16  2.0  5.0  13.208380  1
10  4.0  1.0  10.884523  1
2   6.0  5.0  10.712853  1
3   7.0  7.0  13.032182  1
0   3.0  4.0   8.643989  1
11  6.0  8.0  11.547413  1
7   8.0  3.0  14.015611  0
19  7.0  5.0  10.397102  1
25  7.0  6.0   8.370504  1
26  2.0  8.0   9.254854  1
14  8.0  2.0   9.374562  1
20  3.0  6.0  10.774097  0
15  7.0  6.0  10.463312  1
21  8.0  6.0   9.125668  1
1   6.0  7.0  10.572764  1
6   8.0  6.0  11.311160  1
8   2.0  6.0   9.631790  0
5   6.0  6.0   8.614768  0

## Splitting a dataset into training and test datasets

Another useful method provided by the above dataset class is the `train_test_split` method. This method splits the dataset into a training and test sets. Here is how this method is implemented:

```python
    ...
    def train_test_split(self,start=0, end=None, test_portion=None, shuffle=False, random_state=None):
        """
        Splits the dataset into a training set and atest set. 
        If test_portion is specified, return that portion of the dataset as test 
        and the rest as training. 
        Otherwise, return the examples between start and end as test and the 
        rest as training.
        """
        indexes = np.arange(self.N)
        if shuffle is True:
            rgen = np.random.RandomState(random_state)
            rgen.shuffle(indexes)

        if test_portion is None:
            end = end or self.N
        else:
            if not isinstance(test_portion, float) or test_portion < 0 or test_portion > 1:
                raise TypeError("Only fractions between ]0,1[ are allowed")

            start = self.N - int(self.N * test_portion)
            end = self.N

        test = DataSet(self.examples.iloc[indexes[range(start, end)]])
        train = DataSet(pd.concat([self.examples.iloc[indexes[range(start)]], 
                                      self.examples.iloc[indexes[range(end, self.N)]]], axis=0))    
        return train, test
   ...
```

If the `start` and end `end` parameters exist, the method returns the examples before them as test and the rest of the data as training. If `test_portion` is provided, then that portion of the data is returned as test and the rest as training. The `shuffle` parameter can be used to instruct the method to shuffle the data before splitting it. The method finally returns two dataset instances: training and test sets.

Here is an example using this method.

In [223]:
ds = DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")


ta, te = ds.train_test_split(test_portion=.25, shuffle=False, random_state=17)
print('Training set = \n', ta)
print('Test set = \n', te)

Training set = 
      x1   x2         x1  y
0   5.0  7.0   8.496823  1
1   8.0  3.0   6.747611  0
2   2.0  3.0  10.586959  0
3   3.0  6.0   6.277010  0
4   8.0  2.0  12.822289  1
5   8.0  6.0  13.917020  0
6   4.0  6.0   8.739914  0
7   3.0  2.0   7.054237  0
8   8.0  2.0   8.480895  0
9   6.0  6.0   8.286736  1
10  6.0  4.0   7.701750  1
11  7.0  8.0   7.546767  0
12  4.0  3.0  10.154592  0
13  2.0  8.0   9.552458  0
14  6.0  7.0   9.280717  1
15  4.0  5.0  10.573740  1
16  8.0  1.0   9.329067  1
17  6.0  6.0  14.412010  1
18  4.0  8.0   8.707997  0
19  5.0  8.0  11.761178  0
20  8.0  2.0   9.873130  1
Test set = 
      x1   x2         x1  y
21  6.0  3.0  10.139942  0
22  3.0  3.0  12.808626  0
23  3.0  4.0   8.459095  0
24  2.0  7.0   8.628318  0
25  3.0  5.0  11.841330  1
26  5.0  7.0   9.569879  0


## Using this dataset class inside other notebooks

This class is part of the `mylib` library of this class with is provided to you. Here is how to import this library:

In [224]:
import mylib as my

Once imported, one can use it like this:

In [225]:
ds = my.DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")


ta, te = ds.train_test_split(test_portion=.25, shuffle=False, random_state=17)
print('Training set = \n', ta)
print('Test set = \n', te)

Training set = 
      x1   x2         x1  y
0   4.0  3.0   9.537514  0
1   8.0  4.0   9.097263  1
2   6.0  1.0  10.025643  0
3   6.0  4.0   9.983122  0
4   5.0  5.0  10.131466  0
5   7.0  2.0  12.170553  0
6   8.0  8.0  11.279897  1
7   8.0  7.0   5.985314  1
8   7.0  1.0   9.072914  1
9   2.0  6.0  10.804906  0
10  6.0  6.0   9.683898  0
11  6.0  5.0  10.608284  1
12  4.0  5.0  13.687012  1
13  5.0  7.0  14.123362  1
14  4.0  3.0   7.470952  0
15  3.0  7.0  10.882612  1
16  8.0  8.0   9.523496  0
17  4.0  1.0  11.452143  1
18  8.0  7.0   6.644484  1
19  4.0  4.0  13.139126  0
20  6.0  6.0  11.266391  1
Test set = 
      x1   x2         x1  y
21  6.0  6.0   8.853505  0
22  5.0  3.0   7.094571  0
23  8.0  8.0  11.049881  1
24  7.0  8.0   8.674743  0
25  5.0  2.0   6.895826  1
26  5.0  2.0  10.404187  1


## EXERCISE

Refactor the above DataSet class by adding a method named `train_validation_test_split` to it. This method should split the data into three sets: training, validation, and test. This method should receive a dictionary parameter named `portions` specifying how much of the data is in each set. For a 75%/15%/10% split, one can use the following portions parameter:

```python
portions={"training": .75, 'validation': .15, 'test': .10 }
```

The method should support the `shuffle` parameter as well. You may call the `train_test_split` method internally. Make sure to include a comment describing how your implementation of the method works. Test your method on the `ds` dataset above and show that it works.

In [226]:
ds = DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")


ta, vd, te = ds.train_validation_test_split(training=.7, validation=.2, test=.1, shuffle = True)
print('Training set =', ta.N, '\n', ta)
print('Validation =', vd.N, '\n', vd)
print('Test set =', te.N, '\n', te)

Training set = 19 
      x1   x2         x1  y
12  5.0  3.0  13.437295  1
11  5.0  7.0   9.166552  0
18  3.0  3.0  11.384981  0
15  8.0  1.0  11.490866  1
24  3.0  3.0  14.974799  0
10  6.0  6.0   9.587356  0
16  8.0  7.0   8.326838  1
2   2.0  7.0  10.672654  1
8   6.0  6.0   9.396261  0
14  4.0  1.0  13.027683  1
19  3.0  7.0  11.514370  0
20  3.0  8.0   9.255425  1
21  7.0  7.0  13.712035  0
6   4.0  8.0   7.787604  0
26  4.0  3.0   9.208009  0
3   6.0  7.0   9.011952  0
5   5.0  8.0   7.234790  1
7   6.0  3.0  12.522160  0
23  6.0  6.0   8.506949  1
Validation = 5 
      x1   x2         x1  y
17  2.0  3.0   8.287687  0
1   4.0  4.0   6.823321  1
0   5.0  6.0  10.789067  1
9   4.0  3.0  12.793830  0
4   4.0  3.0  11.700366  1
Test set = 3 
      x1   x2         x1  y
13  7.0  4.0  12.308842  0
22  8.0  5.0  10.471603  1
25  4.0  1.0   5.484713  0
